[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nwcallahan/quick-html/blob/main/HTML%20Formatter.ipynb)

# quick-html
Generate simple HTML files from an interactive Jupyter notebook

This tool allows you to interactively write and view a simple HTML file consisting of a title, header, image, and paragraph. The contents of the paragraph is HTML itself, meaning that any typical HTML tags written inside will hold their same properties. Class names and IDs are automatically generated for the built in tags, so a common style.css file can provide formatting that will not alter the contents of the generated file but could clean up the presentation by a large margin.

Ideally the file is run in a local Jupyter notebook, which requires Python to be installed on your machine. However, it can be run online in Google Colaboratory environment (which is just a different format of a Jupyter server) if you login to a Google account. The main disadvantage to this is the code won't be able to automatically save your HTML output & image to your person machine, however by downloading the contents saved on Google (either to your machine or into a Google Drive) you can transfer the code generated. The generated HTML is viewable through the notebook, so copy/pasting is fair game as well.

The notebook should be run cell by cell, attempting to download the HTML with the code in the final cell will result in a blank document if that final cell was not run after changes were added in the prior interactive cells. The final cell is only meant for use with Google Colab, and may be quite buggy. It will not download desired images, only the HTML, but the <img> tag will still be updated according to the URL or local file path you specified prior, so ensuring the link is active or downloading said image to the desired directory will be necessary if accessing online.

If this is adopted it can be easily expanded and formatted, if anyone wants to mess around with a copy be my guest, the HTML template is done with [Airium](https://pypi.org/project/airium/) which is little more than a translation between Python syntax and HTML tags, so modifying the body of the document or ordering is analagous to what would be done within raw HTML.

For convenience, '\n' and '\t' characters in the paragraph body will automatically be converted to HTML versions of a newline and 4 space tab. Images are expected to be sourced directly from URLs, but linking a downloaded image to the HTML should work as well when viewing the final product _outside_ the Jupyter notebook. For whatever reason, even absolute paths to images do not seem to display within the notebook's HTML preview. An option exists to download the images linked by external URL, which saves them to the decided directory alongside the HTML file *and* replaces the <img> tag's source in the HTML code. This is to protect yourself against links rotting between writing and viewing of the resulting webpage, but is not necessary and can cause confusion with the movement of files. *When using this feature, it is expected that downloaded images are stored directly alongside the HTML document.*

In [ ]:
!pip install airium
!pip install ipywidgets

In [1]:
import contextlib, io
zen = io.StringIO()

with contextlib.redirect_stdout(zen):
    import this

zen_of_python = repr(zen.getvalue())

In [2]:
from IPython.display import display, HTML, Image, clear_output
from airium import Airium
import ipywidgets as widgets
import urllib.request
import os
# For HTML Formatting See...
# https://www.w3schools.com/html/html_formatting.asp
result_dict = {
    'Directory' : os.path.join(os.getcwd(), '/CS403Webpage/'),
    'Title' : 'Python Webpage',
    'Header' : 'Python CS403',
    'Paragraph' : zen_of_python.replace(r'\n','<br>').replace('"',''),
    'Image' : 'https://logos-world.net/wp-content/uploads/2021/10/Python-Symbol.png',
}
html = ''
save_path = ''
file_path = ''
page_name = 'webpage.html'

width = '50%'
auto_save = True
img_stored = 'NotIncluded'

In [5]:
directory_box = widgets.Text(
    value=result_dict['Directory'],
    description='Directory:',
    tooltip='Destination Directory for Generated Files',
    layout=widgets.Layout(width=width),
)
display(directory_box)
title_box = widgets.Text(
    value=result_dict['Title'],
    placeholder='Tab Title Here',
    description='Title:',
    layout=widgets.Layout(width=width),
)
display(title_box)
header_box = widgets.Text(
    value=result_dict['Header'],
    placeholder='Text Header Here',
    description='Header:',
    layout=widgets.Layout(width=width),
)
display(header_box)
paragraph_box = widgets.Textarea(
    value=result_dict['Paragraph'],
    placeholder='Formatted Text Here',
    description='Paragraph:',
    layout=widgets.Layout(width=width),
)
display(paragraph_box)
image_box = widgets.Text(
    value=result_dict['Image'],
    placeholder='Link to Image Here',
    description='Image URL:',
    layout=widgets.Layout(width=width),
)
display(image_box)
store_button = widgets.Button(
    value=False,
    description='Store Image',
    button_style='',
    tooltip='Store Image in Local Directory?',
    layout=widgets.Layout(width=width),
)
save_button = widgets.Button(
    value=False,
    description='Save Input',
    button_style='warning',
    tooltip='Update Webpage Document?',
    layout=widgets.Layout(width=width),
)
save_input = widgets.Output()
show_button = widgets.Button(
    value=False,
    description='Show Document',
    button_style='danger',
    tooltip='Show Webpage Results?',
    layout=widgets.Layout(width=width),
)    
show_output = widgets.Output()
finish_button = widgets.Button(
    value=False,
    description='Store Webpage',
    button_style='info',
    tooltip='Show Webpage Results?',
    layout=widgets.Layout(width=width),
)    
finish_output = widgets.Output()
img_output = widgets.Output()

show_text_button = widgets.Button(
    value=False,
    description='Show Raw HTML',
    button_style='success',
    tooltip='Display the HTML Contents?',
    layout=widgets.Layout(width=width),
)    
show_text_output = widgets.Output()

display(widgets.VBox([store_button, img_output]))
display(widgets.VBox([save_button, save_input]))
display(widgets.VBox([show_button, show_output]))
display(widgets.VBox([show_text_button, show_text_output]))
display(widgets.VBox([finish_button, finish_output]))

def finish(b):
    with finish_output:
        clear_output()
        file_path = os.path.join(result_dict['Directory'], page_name)
        with open(file_path, 'w') as f:
            f.write(html)
        print(f"File Written to {file_path} Finished!")

def show_raw_html(b):
    with show_text_output:
        if auto_save: update_dict()
        clear_output()
        print(html)
        return
    
def update_dict():
    result_dict['Directory'] = directory_box.value
    result_dict['Title'] = title_box.value
    result_dict['Header'] = header_box.value
    paragraph_string = paragraph_box.value
    paragraph_string = paragraph_string.replace(r'\t','&emsp;').replace(r'\n','<br>')
    result_dict['Paragraph'] = paragraph_string
    if img_stored not in image_box.value: result_dict['Image'] = image_box.value
    local_path = os.path.join(result_dict['Directory'], result_dict['Image'].split('/')[-1])
    return

def on_click(b):
    with save_input:
        clear_output()
        if auto_save: update_dict()
        print("Results Saved!")

def save_img(b):
    with img_output:
        global img_stored
        clear_output()
        local_path = image_box.value.split('/')[-1]
        abs_path = os.path.join(result_dict['Directory'], local_path)
        target_dir = result_dict['Directory']
        if not os.path.exists(target_dir):
            print(f'Creating Directory {target_dir}')
            os.mkdir(target_dir)
        print(f'Saving Image to {abs_path}')
        if image_box.value.startswith(('htt','www')):
            urllib.request.urlretrieve(image_box.value, abs_path)
        result_dict['Image'] = abs_path.split('/')[-1]
        img_stored = local_path
        update_dict()
        
def show_results(b):
    with show_output:
        global html
        if auto_save: update_dict()
        clear_output()
#         img_path = result_dict['Image'] if img_stored not in result_dict['Image'] else img_stored
        a = Airium()
        a('<!DOCTYPE html>')
        with a.html(lang="pl"):
            with a.head():
                a.meta(charset="utf-8")
                a.link(rel='stylesheet', href='style.css')
                a.title(text=result_dict['Title'])
            with a.body():
                with a.h1(id='1', klass='main_header'):
                    a(result_dict['Header'])
            a.img(src=result_dict['Image'], id='3', klass='main_image', alt='Image Should be Stored Locally', width='25%', height='auto')
            with a.body():
                with a.p(id='2', klass='main_paragraph'):
                    a(result_dict['Paragraph'])
        # Casting the file to a string to extract the value
        html = str(a)
        display(HTML(html))
        result_dict['HTML'] = html
        return
    
show_text_button.on_click(show_raw_html)
store_button.on_click(save_img)
show_button.on_click(show_results)    
save_button.on_click(on_click)
finish_button.on_click(finish)

Text(value='C:/CS403Webpage/', description='Directory:', layout=Layout(width='50%'))

Text(value='Python Webpage', description='Title:', layout=Layout(width='50%'), placeholder='Tab Title Here')

Text(value='Python CS403', description='Header:', layout=Layout(width='50%'), placeholder='Text Header Here')

Textarea(value="The Zen of Python, by Tim Peters<br><br>Beautiful is better than ugly.<br>Explicit is better t…

Text(value='C:/CS403Webpage/Python-Symbol.png', description='Image URL:', layout=Layout(width='50%'), placehol…

In [4]:
import base64

#FILE
b64 = base64.b64encode(result_dict['HTML'].encode())

html_buttons = f'''
<html>
    <head>
        <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
        <a download="{page_name}" href="data:text/csv;base64,{b64.decode()}" download>
            <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download {page_name}</button>
        </a>
    </body>
</html>
'''

# html_button = html_buttons.format(payload=b64.decode(), filename=filename)
display(HTML(html_buttons))